In [1]:
import pandas as pd
import os

In [2]:
def get_list_source_target(lang='EN'):
    list_dir = os.listdir('../Small/')
    list_dir_en_source = [x for x in list_dir if (lang+'-') in x]
    list_dir_en_target = [x for x in list_dir if ('-'+lang) in x]
    return list_dir_en_source, list_dir_en_target

In [3]:
langs = ['PT', 'AN', 'RO', 'SC', 'FR', 'CA', 'IT', 'EN', 'EU', 'EO', 'ES', 'OC', 'GL']
def get_lang_src_target(path):
    src_target = path.split('_')[0].split('Set')[1].split('-')
    return src_target
def get_list_source_target_infe(lang='EN'):
    list_dir = os.listdir('../Small/')
    list_dir = [x for x in list_dir if '.csv' in x]
    list_dir_en_source = [x for x in list_dir if get_lang_src_target(x)[0]==lang and get_lang_src_target(x)[1] in langs]
    list_dir_en_target = [x for x in list_dir if get_lang_src_target(x)[1]==lang and get_lang_src_target(x)[0] in langs]
    return list_dir_en_source, list_dir_en_target

In [4]:
get_list_source_target_infe()

(['TransSetEN-GL_small.csv',
  'TransSetEN-ES_small.csv',
  'TransSetEN-CA_small.csv'],
 ['TransSetEU-EN_small.csv', 'TransSetEO-EN_small.csv'])

In [6]:
def get_data_frame_lang(lang='EN'):
    df_en= pd.DataFrame(columns=['source', 'POS'])
    source, target = get_list_source_target(lang)
    for path_source in source:
      path_source = '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['source', 'POS']]
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    for path_source in target:
      path_source =  '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['target', 'POS']]
      df_form_source.columns = ['source', 'POS']
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    return df_en

In [7]:
for lang in ['EN', 'FR', 'PT']:
    df = get_data_frame_lang(lang)
    df = df.drop_duplicates(ignore_index=True)
    df = df.sort_values(by=['POS'], ignore_index=True)
    path = '../Small/'+lang+'_vocab/'
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    
    df.to_csv(path+'all.csv', index=False)
    max_rows = 4500
    dataframes = []
    while len(df) > max_rows:
        top = df[:max_rows]
        dataframes.append(top)
        df = df[max_rows:]
    else:
        dataframes.append(df)
    for _, frame in enumerate(dataframes):
        frame.to_csv(path+str(_)+'.csv', index=False)

Creation of the directory ../Small/EN_vocab/ failed
Creation of the directory ../Small/FR_vocab/ failed
Creation of the directory ../Small/PT_vocab/ failed


In [22]:
from IPython.display import display
list_in = ['properNoun', 'noun', 'adjective', 'verb', 'adverb', 'preposition', 'determiner', 'numeral']

In [66]:
df = get_data_frame_lang()
df = df.drop_duplicates(ignore_index=True)
count = df['POS'].value_counts().to_frame().reset_index()
count_en = count[count['index'].isin(list_in)]

df = get_data_frame_lang('ES')
df = df.drop_duplicates(ignore_index=True)
count = df['POS'].value_counts().to_frame().reset_index()
count_es = count[count['index'].isin(list_in)]

df = get_data_frame_lang('FR')
df = df.drop_duplicates(ignore_index=True)
count = df['POS'].value_counts().to_frame().reset_index()
count_fr = count[count['index'].isin(list_in)]

df = get_data_frame_lang('PT')
df = df.drop_duplicates(ignore_index=True)
count = df['POS'].value_counts().to_frame().reset_index()
count_pt = count[count['index'].isin(list_in)]

df = get_data_frame_lang('CA')
df = df.drop_duplicates(ignore_index=True)
count = df['POS'].value_counts().to_frame().reset_index()
count_ca = count[count['index'].isin(list_in)]

In [79]:
from functools import reduce
# count_1.merge(count_2, on='index', suffixes=('_EN', '_FR')).merge(count_3, on='index', suffixes=('_EN', '_ES'))

In [80]:
list_dir = os.listdir('../Small/')
list_dir = [x for x in list_dir if x.endswith('.csv')]

df_all= pd.DataFrame(columns=['source', 'POS'])
for path_source in list_dir:
      path_source = '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['source', 'POS']]
      df_all = pd.concat([df_all, df_form_source], ignore_index=True)
df_all = df_all.drop_duplicates(ignore_index=True)
count = df_all['POS'].value_counts().to_frame().reset_index()
count_all = count[count['index'].isin(list_in)]

In [81]:
data_frames = [count_all, count_en, count_pt, count_fr, count_ca, count_es]
reduce(lambda left,right: pd.merge(left,right,on='index'), data_frames)

,index,POS_x,POS_y,POS_x,POS_y,POS_x,POS_y
0,noun,307599,26707,11475,27798,33050,35829
1,properNoun,166166,43856,24210,36626,42573,34225
2,adjective,107067,9990,9670,11519,17100,19939
3,verb,70248,6640,2519,6208,6574,9015
4,adverb,44566,4756,2556,3670,8010,9556
5,preposition,3819,263,133,394,419,464
6,numeral,2300,265,162,220,229,236
7,determiner,1487,216,30,67,100,154


In [8]:
all_en = pd.read_csv('../Small/EN_vocab/all.csv')
all_fr = pd.read_csv('../Small/Fr_vocab/all.csv')
all_pt = pd.read_csv('../Small/PT_vocab/all.csv')

In [9]:
print('len en: {},len fr: {}, len pt: {}'.format(len(all_en), len(all_fr), len(all_pt)))

len en: 93466,len fr: 87146, len pt: 51035


In [10]:
check_en_fr = pd.merge(all_en, all_fr, on=['source', 'POS'], how='inner')

In [11]:
all_en.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]

,source,POS
4,adolescent,adjective
9,militant,adjective
17,compact,adjective
28,adjectival,adjective
29,changeable,adjective
...,...,...
89135,flatter,verb
89998,layer,verb
90286,solder,verb
92090,twitter,verb


In [12]:
def transfer_same(same):
    same['confidence'] = 1.0
    same['target'] = same['source']
    same = same[['source','target', 'POS', 'confidence']]
    return same

In [14]:
#get difference between en_fr
#all_en.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_en_fr.csv', index=False)

same_en_fr = all_en.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]
same_en_fr = transfer_same(same_en_fr)
# same_en_fr.to_csv('../Small/Dictionary_Same/same_en_fr.tsv', index=False, sep='\t', header=False)
same_en_fr[same_en_fr['POS']=='comma']

,source,target,POS,confidence
14748,NaN,NaN,comma,1.0


In [64]:
#get difference between en_pt
all_en.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_en_pt.csv', index=False)
#get difference same en_pt
same_en_fr = all_en.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]
same_en_fr = transfer_same(same_en_fr)
same_en_fr.to_csv('../Small/Dictionary_Same/same_en_pt.tsv', index=False, sep='\t', header=False)

In [66]:
#get difference between fr_en
#all_fr.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_fr_en.csv', index=False)

same = all_fr.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]
same = transfer_same(same)
same.to_csv('../Small/Dictionary_Same/same_fr_en.tsv', index=False, sep='\t', header=False)

In [67]:
#get difference between fr_pt
all_fr.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_fr_pt.csv', index=False)
#get difference same fr_pt
same = all_fr.merge(all_pt,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]
same = transfer_same(same)
same.to_csv('../Small/Dictionary_Same/same_fr_pt.tsv', index=False, sep='\t', header=False)

In [68]:
#get difference between pt_en
all_pt.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_pt_en.csv', index=False)
#get difference same pt_en
same=all_pt.merge(all_en,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]
same = transfer_same(same)
same.to_csv('../Small/Dictionary_Same/same_pt_en.tsv',  index=False, sep='\t', header=False)

In [69]:
#get difference between pt_fr
all_pt.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_Difference/diff_pt_fr.csv', index=False)
#get difference same pt_fr

same = all_pt.merge(all_fr,indicator = True, how='left').loc[lambda x : x['_merge']=='both'][['source', 'POS']]
same = transfer_same(same)
same.to_csv('../Small/Dictionary_Same/same_pt_fr.tsv', index=False, sep='\t', header=False)

In [83]:
def get_data_frame_lang_infe(lang='EN'):
    df_en= pd.DataFrame(columns=['source', 'POS'])
    source, target = get_list_source_target_infe(lang)
    for path_source in source:
      path_source = '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['source', 'POS']]
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    for path_source in target:
      path_source =  '../Small/' + path_source 
      df = pd.read_csv(path_source, delimiter=',', error_bad_lines=False)
      df_form_source = df[['target', 'POS']]
      df_form_source.columns = ['source', 'POS']
      df_en = pd.concat([df_en, df_form_source], ignore_index=True)
    return df_en

In [91]:
df_en_infe = get_data_frame_lang_infe().drop_duplicates(ignore_index=True)
df_fr_infe = get_data_frame_lang_infe('FR').drop_duplicates(ignore_index=True)
df_pt_infe = get_data_frame_lang_infe('PT').drop_duplicates(ignore_index=True)

In [92]:
print(df_en_infe.count())
print(df_fr_infe.count())
print(df_pt_infe.count())

source    59576
POS       59578
dtype: int64
source    82200
POS       82201
dtype: int64
source    51035
POS       51035
dtype: int64


In [95]:
count = df_en_infe['POS'].value_counts().to_frame().reset_index()
count_en = count[count['index'].isin(list_in)]

count = df_fr_infe['POS'].value_counts().to_frame().reset_index()
count_fr = count[count['index'].isin(list_in)]

count = df_pt_infe['POS'].value_counts().to_frame().reset_index()
count_pt = count[count['index'].isin(list_in)]

In [96]:
data_frames = [ count_en, count_pt, count_fr]
reduce(lambda left,right: pd.merge(left,right,on='index'), data_frames)

,index,POS_x,POS_y,POS
0,properNoun,20978,24210,36626
1,noun,19945,11475,27798
2,adjective,7996,9670,11519
3,verb,5299,2519,6208
4,adverb,4266,2556,3670
5,preposition,212,133,394
6,determiner,193,30,67
7,numeral,168,162,220


In [18]:
# create difference word beetween 2 dictionary
df_en_infe.merge(df_fr_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_en_fr.csv', index=False)
df_en_infe.merge(df_pt_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_en_pt.csv', index=False)

df_fr_infe.merge(df_en_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_fr_en.csv', index=False)
df_fr_infe.merge(df_pt_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_fr_pt.csv', index=False)

df_pt_infe.merge(df_en_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_pt_en.csv', index=False)
df_pt_infe.merge(df_fr_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_pt_fr.csv', index=False)

In [20]:
# create same word beetween 2 dictionary
df_en_infe.merge(df_fr_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_en_fr.csv', index=False)
df_en_infe.merge(df_pt_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_en_pt.csv', index=False)

df_fr_infe.merge(df_en_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_fr_en.csv', index=False)
df_fr_infe.merge(df_pt_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_fr_pt.csv', index=False)

df_pt_infe.merge(df_en_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_pt_en.csv', index=False)
df_pt_infe.merge(df_fr_infe,indicator = True, how='left').loc[lambda x : x['_merge']!='both'][['source', 'POS']].to_csv('../Small/Dictionary_To_Infe/to_infe_pt_fr.csv', index=False)

,source,POS
19,apparition,noun
20,speculation,noun
40,interposition,noun
45,tribal,adjective
57,restriction,noun
...,...,...
59405,afghan,noun
59406,procrastination,noun
59458,national,noun
59479,passable,adjective
